# N-Grams

N-Grams allow us to calculate probabilitties of certain words occouring in a specific sequence. This can be used to build a
- Auto-Complete Model
- Search Sugesstion Tool

An N-gram is a squence of N words (punctuation is treated as words)

Corpus : I am happy because I am learning
Unigram : {I, am, happy, because, learning}
Bigram : {I am, am happy, happy because, ...}
Trigrams : {Similarly Tripletes ...}

### Unigram Probability
Sort of obvious, no?

### Bigram and N-Gram Probability

P(am|I) = C(I am)/C(I)

Similarly, a tri-gram probability

P(w3 | w1^2) = C(w1^2w3)/C(w1^2)

### How to model whole sentence using N-Gram Probability

Use Chain Rule, 
P(A,B,C,D) = P(A) P(B|A) P(C|A,B) P(D|A,B,C)
Now, to calcualte the probability each of these elements must have a pre assigned probability, which is very diffculat to assertain, as the number of words increases.

So, in order to ease this, we simply use an approximation
Markoff Assumption : The probability of a word depends only on some of its predecessors, and not all.

thus,
P(A,B,C,D) = P(A) P(B|A) P(C|B) P(D|B)

### How to get this N-Gram Probability using a corpus?

We use two new symbols to denote, (start  angle_brackets(s) and the end of sentences angle_brackets(/s)).

add N-1 start tokens, for an N-Gram.

Why do we need the end-token?

Eg: 
angle_brackets(s) yes no
angle_brackets(s) yes yes
angle_brackets(s) no no

Consider this corpus.

P("angle_brackets(s)" yes yes) = P(yes | angle_brackets(s)) P(yes | yes) = 2/3 * 1/3
P(angle_brackets(s) yes no) = 1/3
P(angle_brackets(s) no no) = 1/3
P(angle_brackets(s) no yes) = 0

Sum of this is 1

Considier sentences of length 3,
P(angle_brackets(s) yes yes yes) = ...

Sum of this is also 1.


What do we want? We want the sum of probabilties of all, two, three ... lenght sentences to be equal to one (taken all at once, not individually). Thus, we add end tokens.


## How to populate and create a count matrix
This matrix tells, how many times a word is followed by another word.
Then we transform the count matrix to a probability matrix.

# Perplexity

This is a metric used to evaluate a language model. [80 10 10][Train Validation Test].

### How to split the data?
- Continous Text
- Short Sequence of words, such as sentences.

Perplexity is the measure of the complexity in a sample of text. It is used to tell us, if a set of sentences was written by humans, rather than by a simple program. [A text written by a human will have less perxplexity] [Closely realte to entropy]


# Out Of Vocabulary Word
These are words, that are outside the vocabulary (set of fixed questions, that are used for the alngauge model)

Unknown Words are known as out of vocabulary words.
Replace these words by \<UNK\>

- Words, that become 

# Smoothing

- It is not necessary, that every combinarion of two words will appear in the bigram (similarly N-gram). To solve this we use N-grams.

add, one occourance to each. (instead, add k for a larger corpus)

- Another approach, is Backoff i.e, using lower level n-grams. (eg. Katz backoff)
- Stupid Backoff is another methods


- Interpolation, can also be used.  [Interpolation constants can be learned from the validation test.]



